In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pathlib
model_dir = pathlib.Path().resolve().parent/"models"
model_name = "Qwen/Qwen2.5-0.5B-Instruct"\

model_dir

WindowsPath('C:/Users/egor/files/projects/FPA_NLP/models')

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    cache_dir=model_dir
)



c:\Users\egor\files\projects\FPA_NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\egor\files\projects\FPA_NLP\models\models--Qwen--Qwen2.5-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
model = model.to("cuda")
model.device

device(type='cuda', index=0)

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
prompt = """ Расскажи суть следующего диалога
А. – Алло. Центр «Интерконгресс» .

Б. – Здравствуйте. Радитехнический университет. Миронова Ольга. Я вам вчера звонила относительно симпозиума.

А. – Добрый день. Слушаю вас.

Б. – Вы можете предоставить нам помещение на сто тридцать – сто сорок человек?

А. – Да. Мы можем забронировать для вас конференц-зал на сто пятьдесят мест.

Б. – Это нам подходит, спасибо. А как оформить аренду мебели?

А. – В заявке нужно точно указать названия всех предметов и их количество.

Б. – Как отправить вам заявку?

А. – Вы должны отправить на наш адрес гарантийное письмо. В нем вы и укажете все виды услуг и их стоимость.

Б. – Понятно. Письмо можно отправить и по почте, и по факсу.

А. – Да.

Б. – И через сколько дней вы его получите?

А. – Письмо идет обычно два-три дня.

Б. – Это довольно долго.

А. – Можно послать с нарочным, тогда оно придет к нам в течение двух часов.

Б. – Так мы и сделаем. Спасибо большое за исчерпывающую информацию.

А. – Пожалуйста. Всего доброго."""
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

In [17]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [18]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

In [20]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

In [21]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)